### MNIST DNN with Estimators & Layers

In this example, we will use a deep neural network to classify the MNIST dataset. This code is very similar to the 5th example (deep_neural_network_mnist_layers), however instead of manually managing the session and TensorBoard, we let Estimators handle this for us.

In [18]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import numpy as np
import os

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

When using Estimators, we do not manage the TensorFlow session directly. Instead, we skip straight to defining our hyperparameters.

In [19]:
# Number of neurons in each hidden layer
HIDDEN1_SIZE = 500
HIDDEN2_SIZE = 250

NUM_CLASSES = 10  # 10 digits 0-9
NUM_PIXELS = 28 * 28  # dataset size.

# Experiment with the number of training steps to see the effect.
TRAIN_STEPS = 2000
BATCH_SIZE = 100

# We're using a different learning rate than the previous notebook, and a new optimizer
LEARNING_RATE = 0.001

In [64]:
# Define our model
def model_fn(features, labels, mode):
    
    # First we'll create 2 fully-connected layers, with ReLU activations.
    fc1 = tf.layers.dense(features['x'], HIDDEN1_SIZE, activation=tf.nn.relu, name="fc1")
    fc2 = tf.layers.dense(fc1, HIDDEN2_SIZE, activation=tf.nn.relu, name="fc2")

    # Next, we'll apply Dropout to the second layer
    # This can help prevent overfitting, and I've added it here
    # for illustration. You can comment this out, if you like.
    dropped = tf.nn.dropout(fc2, keep_prob=0.9, name="dropout1")

    # Finally, we'll calculate logists. This will be
    # the input to our Softmax function. Notice we 
    # don't apply an activation at this layer.
    # If you've commented out the dropout layer,
    # switch the input here to 'fc2'.
    y = tf.layers.dense(dropped, NUM_CLASSES, name="output")
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        preds = {
            'class': tf.argmax(y, axis=1),
            'prediction': y
        }
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=preds)
    
    # Compute the loss.
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=labels, logits=y)
    
    tf.summary.scalar('loss', loss)
    
    learning_rate = LEARNING_RATE
    # Alternate learning rate calculation
    # learning_rate = tf.train.exponential_decay(
    # LEARNING_RATE, tf.train.get_global_step(), 100000, 0.96)
    
    # Define the optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    # Return an estimator spec that encapsulates your model.
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss,
                                      train_op=train_op)


In [65]:
# Number of samples for prediction.
NUM_PRED_SAMPLES = 5

# Define our data input function.
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

x_train = np.array(mnist.train.images, dtype=np.float32)
y_train = np.array(mnist.train.labels, dtype=np.int32)

x_test = np.array(mnist.test.images, dtype=np.float32)
y_test = np.array(mnist.test.labels, dtype=np.int32)

mnist_train_input = tf.estimator.inputs.numpy_input_fn({'x': x_train},
                                                       y_train,
                                                       shuffle=True,
                                                       num_epochs=None)

mnist_test_input = tf.estimator.inputs.numpy_input_fn({'x': x_test},
                                                      y_test,
                                                      shuffle=True,
                                                      num_epochs=1)

mnist_sample_pred_input = tf.estimator.inputs.numpy_input_fn({'x': x_test[:NUM_PRED_SAMPLES]},
                                                      y_test[:NUM_PRED_SAMPLES],
                                                      shuffle=False,
                                                      num_epochs=1)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [66]:
# Define the run configuration.
estimator = tf.estimator.Estimator(model_fn=model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_model_dir': '/tmp/tmpmdoqttq7', '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_tf_random_seed': 1}


In [67]:
# Train the estimator using our input function.
estimator.train(input_fn=mnist_train_input, steps=TRAIN_STEPS)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpmdoqttq7/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.31158
INFO:tensorflow:global_step/sec: 164.201
INFO:tensorflow:step = 101, loss = 2.24726 (0.610 sec)
INFO:tensorflow:global_step/sec: 186.516
INFO:tensorflow:step = 201, loss = 2.16199 (0.536 sec)
INFO:tensorflow:global_step/sec: 186.107
INFO:tensorflow:step = 301, loss = 2.07692 (0.537 sec)
INFO:tensorflow:global_step/sec: 196.359
INFO:tensorflow:step = 401, loss = 1.92831 (0.509 sec)
INFO:tensorflow:global_step/sec: 193.51
INFO:tensorflow:step = 501, loss = 1.93974 (0.517 sec)
INFO:tensorflow:global_step/sec: 198.303
INFO:tensorflow:step = 601, loss = 1.85007 (0.504 sec)
INFO:tensorflow:global_step/sec: 197.432
INFO:tensorflow:step = 701, loss = 1.78981 (0.507 sec)
INFO:tensorflow:global_step/sec: 200.727
INFO:tensorflow:step = 801, loss = 1.63758 (0.498 sec)
INFO:tensorflow:global_step/sec: 194.234
INFO:tensorflow:step = 901, loss

In [68]:
# Evaluate the estimator using our input function.
evaluation = estimator.evaluate(input_fn=mnist_test_input)
print(evaluation)

INFO:tensorflow:Starting evaluation at 2017-08-04-22:01:47
INFO:tensorflow:Restoring parameters from /tmp/tmpmdoqttq7/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-08-04-22:01:47
INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.96981
{'global_step': 2000, 'loss': 0.96981019}


In [75]:
# Prediction
preds = estimator.predict(input_fn=mnist_sample_pred_input)
i = 0
for p in preds:
    print('-------------- Test %d -------------' % i)
    print('Prediction:', p['class'])
    print('Label:', np.argmax(y_test[i]))
    i += 1

INFO:tensorflow:Restoring parameters from /tmp/tmpmdoqttq7/model.ckpt-2000
-------------- Test 0 -------------
Prediction: 7
Label: 7
-------------- Test 1 -------------
Prediction: 2
Label: 2
-------------- Test 2 -------------
Prediction: 1
Label: 1
-------------- Test 3 -------------
Prediction: 0
Label: 0
-------------- Test 4 -------------
Prediction: 4
Label: 4
